# This notebook was made to deliver the IBM capstone final project.

### In order to create a dataframe from a csv file that contains all the cities around the world, we use pandas.

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

### The data will be readen from a .csv file obtained from the website: https://simplemaps.com/data/world-cities

In [3]:
df_world = pd.read_csv('datasets/worldcities.csv')
df_world.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


### Now we need to filter our dataframe to retrieve only the cities from the contry "Brazil", which is where we are analysing.

In [4]:
df_brazil = df_world.loc[df_world['country'] == 'Brazil']
df_brazil.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
6,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22046000.0,1076532519
28,Rio de Janeiro,Rio de Janeiro,-22.9083,-43.1964,Brazil,BR,BRA,Rio de Janeiro,admin,12272000.0,1076887657
138,Belo Horizonte,Belo Horizonte,-19.8917,-43.9478,Brazil,BR,BRA,Minas Gerais,admin,5159000.0,1076967355
258,Brasília,Brasilia,-15.7744,-48.0773,Brazil,BR,BRA,Distrito Federal,primary,3015268.0,1076144436
269,Salvador,Salvador,-12.9708,-38.5108,Brazil,BR,BRA,Bahia,admin,2921087.0,1076923789


### The only columns that are interesting for now are the city_asc name, the coordinates its the population.

In [5]:
df_brazil.drop(columns=[
                        'city',
                        'country',
                        'iso2', 
                        'iso3',
                        'admin_name',
                        'capital',
                        'id'
                        ],
                inplace = True)
df_brazil.head()

C:\Users\Rodol\Anaconda3\envs\tensorflow2_gpu\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,city_ascii,lat,lng,population
6,Sao Paulo,-23.5504,-46.6339,22046000.0
28,Rio de Janeiro,-22.9083,-43.1964,12272000.0
138,Belo Horizonte,-19.8917,-43.9478,5159000.0
258,Brasilia,-15.7744,-48.0773,3015268.0
269,Salvador,-12.9708,-38.5108,2921087.0


### Now we will sort by the population value, so that the first rows represents the most populated cities.

In [6]:
df_brazil = df_brazil.sort_values(by='population', ascending=False).reset_index(drop=True)
df_brazil.head()

,city_ascii,lat,lng,population
0,Sao Paulo,-23.5504,-46.6339,22046000.0
1,Rio de Janeiro,-22.9083,-43.1964,12272000.0
2,Belo Horizonte,-19.8917,-43.9478,5159000.0
3,Brasilia,-15.7744,-48.0773,3015268.0
4,Salvador,-12.9708,-38.5108,2921087.0


### Great, now that our dataframe is already ordered by the population, its value will not add valuable information for what we are trying to achive, so it will also be dropped.

In [7]:
df_brazil.drop(columns=['population'],
                inplace = True)
df_brazil.head()

,city_ascii,lat,lng
0,Sao Paulo,-23.5504,-46.6339
1,Rio de Janeiro,-22.9083,-43.1964
2,Belo Horizonte,-19.8917,-43.9478
3,Brasilia,-15.7744,-48.0773
4,Salvador,-12.9708,-38.5108


### I will change the name of the column "city_ascii" to only "city" for simplicity.

In [8]:
df_brazil.rename(columns={'city_ascii': 'city'}, 
                 inplace = True)
df_brazil.head()

,city,lat,lng
0,Sao Paulo,-23.5504,-46.6339
1,Rio de Janeiro,-22.9083,-43.1964
2,Belo Horizonte,-19.8917,-43.9478
3,Brasilia,-15.7744,-48.0773
4,Salvador,-12.9708,-38.5108


### Now that we have the coordinates of each city (lat = latitude, lng = longitude) in a processed dataframe, it is time to get those  into the Folium library, but first, in order to do that, it is necessary to obtain the latitude and longitude values of Brazil.

In [9]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [10]:
address = "Brazil"

geolocator = Nominatim(user_agent="brazil_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Brazil are:\nlatitude: {latitude}\nlongitude: {longitude}.')

The geograpical coordinate of Brazil are:
latitude: -10.3333333
longitude: -53.2.


In [11]:
import folium # map rendering library

In [12]:
# create map of Brazil using the latitude and longitude values obtained
brazil_map = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, city in zip(df_brazil['lat'], df_brazil['lng'], df_brazil['city']):
    label = f'{city}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(brazil_map)  
    
brazil_map

### For the sake of my own privacy, I will not show the credentials that I used.

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [14]:
import requests

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url =f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={radius}&limit={LIMIT}'
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each city and create a new dataframe called brazil_venues.

In [16]:
brazil_venues = getNearbyVenues(names=df_brazil['city'],
                                 latitudes=df_brazil['lat'],
                                 longitudes=df_brazil['lng']
                                 )

Sao Paulo
Rio de Janeiro
Belo Horizonte
Brasilia
Salvador
Fortaleza
Curitiba
Manaus
Vitoria
Recife
Porto Alegre
Goiania
Belem
Guarulhos
Vila Velha
Campinas
Sao Luis
Maceio
Joinvile
Natal
Olinda
Natal
Novo Hamburgo
Campo Grande
Teresina
Iguacu
Sao Bernardo do Campo
Joao Pessoa
Santo Andre
Jaboatao
Osasco
Sao Jose dos Campos
Aracaju
Ribeirao Preto
Sao Jose dos Pinhais
Sorocaba
Uberlandia
Canoas
Cuiaba
Londrina
Niteroi
Feira de Santana
Americana
Florianopolis
Macapa
Vila Velha
Juiz de Fora
Volta Redonda
Maua
Sao Jose do Rio Preto
Santos
Mogi das Cruzes
Diadema
Ipatinga
Campina Grande
Jundiai
Carapicuiba
Campos
Rio Branco
Caxias do Sul
Bauru
Anapolis
Montes Claros
Sao Vicente
Itaquaquecetuba
Piracicaba
Franca
Maringa
Itajai
Boa Vista
Uberaba
Pelotas
Guaruja
Blumenau
Vitoria da Conquista
Porto Velho
Taubate
Praia Grande
Petropolis
Limeira
Ponta Grossa
Crato
Crato
Palmas
Taboao da Serra
Maraba
Sumare
Barueri
Petrolina
Cascavel
Foz do Iguacu
Governador Valadares
Varzea Grande
Santa Maria
Caru

Santa Maria da Serra
Campina do Monte Alegre
Poloni
Fernando Prestes
Coroados
Piacatu
Tres Fronteiras
Guaimbe
Santa Ernestina
Taiuva
Taquarivai
Barra do Chapeu
Alambari
Caiua
Luisiania
Sabino
Aramina
Brauna
Maraba Paulista
Avai
Salmourao
Guzolandia
Alvaro de Carvalho
Ipigua
Pratania
Tonantins
Xinguara
Amapa
Abuna
Vila Bela da Santissima Trindade
Principe da Beira


In [17]:
brazil_venues.shape

(10211, 7)

In [18]:
brazil_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sao Paulo,-23.5504,-46.6339,Casa de Francisca,-23.548733,-46.634763,Music Venue
1,Sao Paulo,-23.5504,-46.6339,Marco Zero Sucos e Lanches,-23.549720,-46.633903,Snack Place
2,Sao Paulo,-23.5504,-46.6339,Delícias Da Vovó,-23.549430,-46.633466,Dessert Shop
3,Sao Paulo,-23.5504,-46.6339,Justino Ribas Fábrica de Jóias,-23.549488,-46.634376,Jewelry Store
4,Sao Paulo,-23.5504,-46.6339,Teatro CCBB,-23.547644,-46.634733,Theater


### Let's find out how many unique categories can be curated from all the returned venues.

In [19]:
number_of_unique_categories = len(brazil_venues['Venue Category'].unique())
print(f'Number of unique categories: {number_of_unique_categories}')

Number of unique categories: 363


### Let's check how many venues were returned for each city.

In [20]:
brazil_venues.groupby(['City'], as_index=False).count()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abaetetuba,14,14,14,14,14,14
1,Abuna,1,1,1,1,1,1
2,Acarau,9,9,9,9,9,9
3,Acu,18,18,18,18,18,18
4,Adamantina,15,15,15,15,15,15
...,...,...,...,...,...,...,...
701,Volta Redonda,18,18,18,18,18,18
702,Votuporanga,33,33,33,33,33,33
703,Xavantes,4,4,4,4,4,4
704,Xinguara,2,2,2,2,2,2


### Analysing each City.

In [21]:
# one hot encoding
brazil_onehot = pd.get_dummies(brazil_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brazil_onehot['City'] = brazil_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [brazil_onehot.columns[-1]] + list(brazil_onehot.columns[:-1])
brazil_onehot = brazil_onehot[fixed_columns]

brazil_onehot.head()

,Zoo,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baiano Restaurant,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Casino,Central Brazilian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dry Cleaner,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Empada House,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Goiano Restaurant,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Knitting Store,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lottery Retailer,Lounge,Mac & Cheese Joint,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,Night Market,Nightclub,Nightlife Spot,Noodle House,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racecourse,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwi

### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category.

In [22]:
brazil_grouped = brazil_onehot.groupby('City').mean().reset_index()
brazil_grouped.head()

,City,Zoo,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Baiano Restaurant,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Car Wash,Casino,Central Brazilian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dry Cleaner,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Empada House,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Goiano Restaurant,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Knitting Store,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lottery Retailer,Lounge,Mac & Cheese Joint,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,Night Market,Nightclub,Nightlife Spot,Noodle House,Northeastern Brazilian Restaurant,Northern Brazilian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racecourse,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Reservoir,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Samba School,Sandwi

In [23]:
brazil_grouped.shape

(706, 363)

### Ok, we can observe from the shape above that it has 367 unique categories (or features), and 706 cities. Now it is possible to investigate which category has a higher frequency along the city.

### Let's print each city along with the top 5 most common venues.

In [24]:
num_top_venues = 5

for hood in brazil_grouped['City']:
    print(f"----{hood}----")
    temp = brazil_grouped[brazil_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abaetetuba----
                           venue  freq
0                       Pharmacy  0.29
1  Paper / Office Supplies Store  0.07
2                      Hotel Bar  0.07
3               Lottery Retailer  0.07
4                            Bar  0.07


----Abuna----
                   venue  freq
0                 Bistro   1.0
1                    Zoo   0.0
2      Outdoor Sculpture   0.0
3  Performing Arts Venue   0.0
4       Pedestrian Plaza   0.0


----Acarau----
                  venue  freq
0              Pharmacy  0.22
1           Coffee Shop  0.11
2     Convenience Store  0.11
3  Fast Food Restaurant  0.11
4           Pizza Place  0.11


----Acu----
         venue  freq
0       Bakery  0.17
1  Snack Place  0.11
2          Bar  0.06
3       Market  0.06
4     Boutique  0.06


----Adamantina----
               venue  freq
0         Restaurant  0.13
1             Bakery  0.13
2     Ice Cream Shop  0.13
3     Nightlife Spot  0.07
4  Convenience Store  0.07


----Aguai----
         

4           Snack Place  0.04


----Bastos----
                venue  freq
0              Bakery  0.33
1  Seafood Restaurant  0.17
2      Ice Cream Shop  0.17
3                Café  0.17
4       Grocery Store  0.17


----Batatais----
            venue  freq
0          Bakery  0.15
1      Restaurant  0.15
2  Ice Cream Shop  0.08
3             Bar  0.08
4        Pharmacy  0.08


----Baturite----
            venue  freq
0           Plaza  0.15
1           Hotel  0.10
2     Pizza Place  0.10
3     Video Store  0.05
4  Gymnastics Gym  0.05


----Bauru----
            venue  freq
0      Restaurant  0.25
1  Soccer Stadium  0.25
2     Pizza Place  0.25
3          Bakery  0.25
4             Zoo  0.00


----Bebedouro----
                  venue  freq
0  Brazilian Restaurant  0.14
1                  Café  0.09
2        Ice Cream Shop  0.09
3           Pizza Place  0.09
4            Restaurant  0.05


----Belem----
                venue  freq
0               Plaza  0.12
1    Department Store  0.09

                  venue  freq
0    Miscellaneous Shop  0.10
1           Supermarket  0.10
2  Gym / Fitness Center  0.10
3                Bakery  0.10
4                 Plaza  0.05


----Campos----
          venue  freq
0  Burger Joint  0.17
1         Plaza  0.08
2           Gym  0.08
3         Hotel  0.08
4        Bistro  0.08


----Campos do Jordao----
              venue  freq
0  Department Store  0.08
1    Sandwich Place  0.08
2       Coffee Shop  0.08
3          Pharmacy  0.08
4       Candy Store  0.04


----Cananeia----
                  venue  freq
0    Seafood Restaurant  0.27
1         Grocery Store  0.13
2                 Beach  0.07
3           Pizza Place  0.07
4  Brazilian Restaurant  0.07


----Canavieiras----
                  venue  freq
0        Ice Cream Shop  0.20
1                 Plaza  0.15
2           Pizza Place  0.10
3  Brazilian Restaurant  0.10
4                 Beach  0.05


----Candido Mota----
            venue  freq
0     Pizza Place  0.12
1     Snack Plac

                  venue  freq
0  Brazilian Restaurant  0.09
1                Bakery  0.06
2         Shopping Mall  0.06
3  Gym / Fitness Center  0.06
4                   Gym  0.06


----Diamantina----
           venue  freq
0          Plaza  0.12
1    Snack Place  0.12
2     Restaurant  0.12
3      BBQ Joint  0.12
4  Grocery Store  0.12


----Divinolandia----
                           venue  freq
0                    Bus Station  0.25
1                     Restaurant  0.25
2                 Clothing Store  0.25
3                          Trail  0.25
4  Northern Brazilian Restaurant  0.00


----Divinopolis----
        venue  freq
0      Bakery  0.31
1  Restaurant  0.12
2      Bistro  0.06
3    Bus Stop  0.06
4         Bar  0.06


----Dobrada----
              venue  freq
0       Snack Place   1.0
1               Zoo   0.0
2      Perfume Shop   0.0
3  Pedestrian Plaza   0.0
4       Pastry Shop   0.0


----Dois Corregos----
              venue  freq
0    Shop & Service  0.17
1  Departmen

                  venue  freq
0                Garden   0.2
1            Restaurant   0.2
2                   Bar   0.2
3        Ice Cream Shop   0.2
4  Brazilian Restaurant   0.2


----Guapiara----
           venue  freq
0         Bakery  0.25
1      BBQ Joint  0.25
2    Bus Station  0.25
3         Garden  0.25
4  Radio Station  0.00


----Guara----
            venue  freq
0     Snack Place   0.2
1  Ice Cream Shop   0.2
2      Restaurant   0.2
3   Grocery Store   0.2
4           Diner   0.2


----Guaracai----
              venue  freq
0    Cricket Ground  0.25
1       Bus Station  0.25
2               Gym  0.25
3  Sushi Restaurant  0.25
4  Pedestrian Plaza  0.00


----Guaraci----
                venue  freq
0  Mineiro Restaurant  0.17
1                 Gym  0.17
2      Ice Cream Shop  0.17
3                Park  0.17
4         Pizza Place  0.17


----Guaranta----
           venue  freq
0  Grocery Store  0.25
1         Bakery  0.12
2          Plaza  0.12
3          Diner  0.12
4    Bus

...

            venue  freq
0          Bakery  0.17
1     Snack Place  0.17
2  Ice Cream Shop  0.17
3    Burger Joint  0.17
4          Market  0.17


----Nhandeara----
                   venue  freq
0         Gymnastics Gym  0.25
1                 Garden  0.25
2                 Market  0.25
3            Pizza Place  0.25
4  Outdoors & Recreation  0.00


----Niquelandia----
                   venue  freq
0                    Bar   0.5
1      Electronics Store   0.5
2                    Zoo   0.0
3  Outdoors & Recreation   0.0
4       Pedestrian Plaza   0.0


----Niteroi----
                   venue  freq
0               Creperie   0.5
1            Cheese Shop   0.5
2      Paella Restaurant   0.0
3           Perfume Shop   0.0
4  Performing Arts Venue   0.0


----Nova Alianca----
                  venue  freq
0  Brazilian Restaurant   0.2
1        Ice Cream Shop   0.2
2                 Plaza   0.2
3             Gift Shop   0.2
4                   Bar   0.2


----Nova Cruz----
               

                  venue  freq
0        Ice Cream Shop  0.12
1  Brazilian Restaurant  0.06
2              Boutique  0.06
3           Snack Place  0.06
4              Pharmacy  0.06


----Penedo----
              venue  freq
0        Restaurant  0.20
1    Clothing Store  0.12
2               Bar  0.08
3  Department Store  0.04
4             Hotel  0.04


----Pereira Barreto----
              venue  freq
0  Asian Restaurant   0.2
1         Gastropub   0.2
2        Restaurant   0.2
3        Food Truck   0.2
4     Grocery Store   0.2


----Pereiras----
                   venue  freq
0            Bus Station  0.25
1                    Bar  0.25
2            Snack Place  0.25
3                    Gym  0.25
4  Outdoors & Recreation  0.00


----Petrolina----
               venue  freq
0         Restaurant  0.67
1        Supermarket  0.33
2                Zoo  0.00
3  Other Repair Shop  0.00
4        Pastry Shop  0.00


----Petropolis----
             venue  freq
0            Hotel  0.15
1  Bed 

                   venue  freq
0                  Plaza  0.25
1          Shopping Mall  0.25
2           Burger Joint  0.25
3                 Bakery  0.25
4  Outdoors & Recreation  0.00


----Ribeirao Branco----
                   venue  freq
0             Restaurant  0.33
1            Music Store  0.33
2                  Hotel  0.33
3                    Zoo  0.00
4  Outdoors & Recreation  0.00


----Ribeirao Pires----
                  venue  freq
0  Brazilian Restaurant  0.07
1   Japanese Restaurant  0.07
2          Burger Joint  0.07
3           Snack Place  0.07
4              Pharmacy  0.07


----Ribeirao Preto----
                  venue  freq
0  Brazilian Restaurant  0.10
1                   Bar  0.09
2        Ice Cream Shop  0.04
3                 Hotel  0.04
4           Coffee Shop  0.04


----Rincao----
           venue  freq
0  Train Station  0.14
1          Plaza  0.14
2            Bar  0.14
3    Pizza Place  0.14
4  Hot Dog Joint  0.14


----Rinopolis----
               ve

                  venue  freq
0  Brazilian Restaurant  0.09
1                Bakery  0.06
2    Italian Restaurant  0.06
3           Pizza Place  0.06
4                   Bar  0.05


----Sao Bento do Sapucai----
                  venue  freq
0  Brazilian Restaurant  0.14
1                Bistro  0.07
2                   Pub  0.07
3       Bed & Breakfast  0.07
4         Historic Site  0.07


----Sao Bernardo do Campo----
                        venue  freq
0                      Bakery  0.11
1               Grocery Store  0.11
2                         Bar  0.05
3  Construction & Landscaping  0.05
4                   Pet Store  0.05


----Sao Borja----
                  venue  freq
0                 Hotel  0.09
1  Brazilian Restaurant  0.06
2             Nightclub  0.06
3                  Park  0.06
4                   Gym  0.06


----Sao Caetano do Sul----
                  venue  freq
0  Brazilian Restaurant  0.12
1                   Spa  0.08
2                   Gym  0.08
3  Gym / Fit

                 venue  freq
0  Dumpling Restaurant   0.2
1          Snack Place   0.2
2        National Park   0.2
3         Dessert Shop   0.2
4          Pizza Place   0.2


----Taruma----
               venue  freq
0  Accessories Store  0.25
1           Pharmacy  0.25
2             Market  0.25
3      Grocery Store  0.25
4                Zoo  0.00


----Tatui----
               venue  freq
0              Plaza  0.14
1  Convenience Store  0.14
2        Snack Place  0.14
3      Grocery Store  0.14
4        Pizza Place  0.14


----Taubate----
                  venue  freq
0  Gym / Fitness Center  0.25
1            Restaurant  0.25
2      Department Store  0.25
3            Food Truck  0.25
4                   Zoo  0.00


----Tefe----
              venue  freq
0  Pedestrian Plaza   1.0
1               Zoo   0.0
2      Perfume Shop   0.0
3       Pastry Shop   0.0
4        Pastelaria   0.0


----Telemaco Borba----
                               venue  freq
0                        Coffee 

### Putting that into a pandas dataframe.

In [25]:
# sorts the venus in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = brazil_grouped['City']

for ind in np.arange(brazil_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brazil_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abaetetuba,Pharmacy,Paper / Office Supplies Store,Hotel Bar,Lottery Retailer,Bar,Convenience Store,Miscellaneous Shop,Public Art,Market,Pizza Place
1,Abuna,Bistro,Zoo,Outdoor Sculpture,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
2,Acarau,Pharmacy,Coffee Shop,Convenience Store,Fast Food Restaurant,Pizza Place,Hunting Supply,Hotel,Market,Paintball Field,Performing Arts Venue
3,Acu,Bakery,Snack Place,Bar,Market,Boutique,Bookstore,Grocery Store,Pastelaria,Gym / Fitness Center,Pet Store
4,Adamantina,Restaurant,Bakery,Ice Cream Shop,Nightlife Spot,Convenience Store,Café,Farmers Market,Chocolate Shop,Pharmacy,Hotel


## Clustering Neighborhoods. Will run k-means to cluster the neighborhood into 5 clusters. For that, it is necessary to import kmeans from scikit-learn

In [28]:
from sklearn.cluster import KMeans

####  The number of clusters was choosen arbitrarily. In the future it will be tested multiple k cluster values as well as adptive algorithms such as DBSCAN and HDBSCAN.

In [29]:
kclusters = 5

brazil_grouped_clustering = brazil_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brazil_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 0, 4, 0, 0, 2, 0])

### Now, let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [30]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brazil_merged = df_brazil

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
brazil_merged = brazil_merged.join(city_venues_sorted.set_index('City'), on='city')

brazil_merged.head()

,city,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sao Paulo,-23.5504,-46.6339,0.0,Brazilian Restaurant,Café,Miscellaneous Shop,Bookstore,Cultural Center,Bakery,Art Gallery,Arts & Crafts Store,Japanese Restaurant,Clothing Store
1,Rio de Janeiro,-22.9083,-43.1964,0.0,Brazilian Restaurant,Restaurant,Bakery,Snack Place,Coffee Shop,Food,Public Art,Music Venue,Electronics Store,Café
2,Belo Horizonte,-19.8917,-43.9478,4.0,Mineiro Restaurant,Ice Cream Shop,Gym / Fitness Center,Fast Food Restaurant,Dessert Shop,Bar,Other Great Outdoors,Park,Perfume Shop,Performing Arts Venue
3,Brasilia,-15.7744,-48.0773,2.0,Bakery,Outdoors & Recreation,Perfume Shop,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
4,Salvador,-12.9708,-38.5108,0.0,Brazilian Restaurant,Plaza,Historic Site,Ice Cream Shop,Hotel,History Museum,Music Venue,Bistro,Vegetarian / Vegan Restaurant,Hostel


In [31]:
city_venues_sorted

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Abaetetuba,Pharmacy,Paper / Office Supplies Store,Hotel Bar,Lottery Retailer,Bar,Convenience Store,Miscellaneous Shop,Public Art,Market,Pizza Place
1,0,Abuna,Bistro,Zoo,Outdoor Sculpture,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
2,0,Acarau,Pharmacy,Coffee Shop,Convenience Store,Fast Food Restaurant,Pizza Place,Hunting Supply,Hotel,Market,Paintball Field,Performing Arts Venue
3,2,Acu,Bakery,Snack Place,Bar,Market,Boutique,Bookstore,Grocery Store,Pastelaria,Gym / Fitness Center,Pet Store
4,0,Adamantina,Restaurant,Bakery,Ice Cream Shop,Nightlife Spot,Convenience Store,Café,Farmers Market,Chocolate Shop,Pharmacy,Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...
701,0,Volta Redonda,Bar,Brazilian Restaurant,Gym / Fitness Center,Seafood Restaurant,Portuguese Restaurant,Burger Joint,Bakery,BBQ Joint,Pizza Place,Playground
702,0,Votuporanga,Plaza,Ice Cream Shop,Japanese Restaurant,Dessert Shop,Pharmacy,Juice Bar,Bar,Sushi Restaurant,Travel & Transport,Sandwich Place
703,0,Xavantes,Steakhouse,Plaza,Department Store,Bus Station,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
704,0,Xinguara,Bar,Convenience Store,Zoo,Outdoors & Recreation,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store



### Finally, let's visualize the resulting clusters. In order to do that, matplotlib will be required. The math module will help identifying if there is any "NaN" along with the data.

In [32]:
import math
import matplotlib.cm as cm
import matplotlib.colors as colors

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brazil_merged['lat'], 
                                  brazil_merged['lng'], 
                                  brazil_merged['city'], 
                                  brazil_merged['Cluster Labels']):
    if math.isnan(float(cluster)) is False:
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    else:
        pass

map_clusters

## Examining Clusters. Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, it is possible to assign a name to each cluster.

#### Cluster 0 is equal to color red.

In [34]:
brazil_merged.loc[brazil_merged['Cluster Labels'] == 0, 
                  brazil_merged.columns[[0] + list(range(4, brazil_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sao Paulo,Brazilian Restaurant,Café,Miscellaneous Shop,Bookstore,Cultural Center,Bakery,Art Gallery,Arts & Crafts Store,Japanese Restaurant,Clothing Store
1,Rio de Janeiro,Brazilian Restaurant,Restaurant,Bakery,Snack Place,Coffee Shop,Food,Public Art,Music Venue,Electronics Store,Café
4,Salvador,Brazilian Restaurant,Plaza,Historic Site,Ice Cream Shop,Hotel,History Museum,Music Venue,Bistro,Vegetarian / Vegan Restaurant,Hostel
5,Fortaleza,Brazilian Restaurant,Shopping Mall,Plaza,Pharmacy,Arts & Crafts Store,Department Store,Women's Store,Bookstore,Electronics Store,Fast Food Restaurant
6,Curitiba,Brazilian Restaurant,Middle Eastern Restaurant,Café,Chocolate Shop,Theater,Snack Place,Plaza,Coffee Shop,Gym / Fitness Center,Burger Joint
...,...,...,...,...,...,...,...,...,...,...,...
772,Sabino,Plaza,Buffet,Beach,Grocery Store,Zoo,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
780,Ipigua,Coffee Shop,Zoo,Outdoors & Recreation,Perfume Shop,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
781,Pratania,Brazilian Restaurant,Leather Goods Store,Museum,Arts & Crafts Store,Northern Brazilian Restaurant,Office,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria
783,Xinguara,Bar,Convenience Store,Zoo,Outdoors & Recreation,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store


#### Cluster 1 is equal to color purple.

In [35]:
brazil_merged.loc[brazil_merged['Cluster Labels'] == 1, 
                  brazil_merged.columns[[0] + list(range(4, brazil_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Pelotas,Steakhouse,Road,Brewery,Buffet,Gymnastics Gym,Playground,Bakery,Restaurant,Public Art,Other Nightlife
76,Taubate,Gym / Fitness Center,Restaurant,Department Store,Food Truck,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park
77,Praia Grande,Pizza Place,Restaurant,Japanese Restaurant,Beer Garden,Brewery,Neighborhood,Gastropub,Beach Bar,Rock Club,Pedestrian Plaza
88,Petrolina,Restaurant,Supermarket,Zoo,Other Repair Shop,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field,Paella Restaurant
123,Passo Fundo,Pedestrian Plaza,Gym,Restaurant,Soccer Field,Outdoor Sculpture,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
...,...,...,...,...,...,...,...,...,...,...,...
743,Colombia,Snack Place,Restaurant,Diner,Hotel,Bar,Plaza,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop
748,Jambeiro,Plaza,Restaurant,South American Restaurant,Zoo,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field,Paella Restaurant
760,Fernando Prestes,Bar,Restaurant,Hotel,Outdoor Sculpture,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
761,Coroados,Brewery,Zoo,Outdoors & Recreation,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field


#### Cluster 2  is equal to color blue.

In [36]:
brazil_merged.loc[brazil_merged['Cluster Labels'] == 2, 
                  brazil_merged.columns[[0] + list(range(4, brazil_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Brasilia,Bakery,Outdoors & Recreation,Perfume Shop,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
19,Natal,Bar,Deli / Bodega,Pastry Shop,Sporting Goods Shop,Bakery,Zoo,Paella Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastelaria
21,Natal,Bar,Deli / Bodega,Pastry Shop,Sporting Goods Shop,Bakery,Zoo,Paella Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastelaria
22,Novo Hamburgo,Pizza Place,Brazilian Restaurant,Burger Joint,Bakery,Zoo,Paella Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria
60,Bauru,Restaurant,Soccer Stadium,Pizza Place,Bakery,Zoo,Paella Restaurant,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria
...,...,...,...,...,...,...,...,...,...,...,...
733,Guaranta,Grocery Store,Bakery,Plaza,Diner,Bus Station,Athletics & Sports,Market,Perfume Shop,Performing Arts Venue,Pedestrian Plaza
735,Tuiuti,Pizza Place,Grocery Store,Bakery,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
751,Pedra Bela,Bakery,Plaza,BBQ Joint,Pizza Place,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
756,Ubarana,Bakery,Outdoors & Recreation,Perfume Shop,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field


#### Cluster 3 is equal to color light green.

In [37]:
brazil_merged.loc[brazil_merged['Cluster Labels'] == 3, 
                  brazil_merged.columns[[0] + list(range(4, brazil_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
600,Tocantins,Snack Place,Gym,Pizza Place,Zoo,Outdoors & Recreation,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
623,Presidente Bernardes,Gym / Fitness Center,Snack Place,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
642,Palmares Paulista,Snack Place,Restaurant,BBQ Joint,Public Art,Market,Zoo,Paella Restaurant,Pedestrian Plaza,Pastry Shop,Pastelaria
655,Iacanga,Snack Place,BBQ Joint,Gym,Bus Station,Zoo,Paella Restaurant,Pedestrian Plaza,Pastry Shop,Pastelaria,Park
656,Areiopolis,Nightclub,Bus Station,Snack Place,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
677,Ouroeste,Snack Place,Convenience Store,Pizza Place,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
681,Herculandia,Snack Place,Gym,Burger Joint,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
698,Dobrada,Snack Place,Zoo,Perfume Shop,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field,Paella Restaurant
704,Pereiras,Bus Station,Bar,Snack Place,Gym,Outdoors & Recreation,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
722,Ibirarema,Snack Place,BBQ Joint,Zoo,Outdoors & Recreation,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field


#### Cluster 4 is equal to color yellow.

In [38]:
brazil_merged.loc[brazil_merged['Cluster Labels'] == 4, 
                  brazil_merged.columns[[0] + list(range(4, brazil_merged.shape[1]))]]

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Belo Horizonte,Mineiro Restaurant,Ice Cream Shop,Gym / Fitness Center,Fast Food Restaurant,Dessert Shop,Bar,Other Great Outdoors,Park,Perfume Shop,Performing Arts Venue
83,Palmas,Pizza Place,Bakery,Ice Cream Shop,Lounge,Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Performing Arts Venue,Pedestrian Plaza
150,Parnaiba,Ice Cream Shop,Pizza Place,Burger Joint,Bakery,Pet Store,Supermarket,Flower Shop,Soccer Field,Gym,Flea Market
175,Pouso Alegre,Bar,Breakfast Spot,Restaurant,Ice Cream Shop,Pizza Place,Mineiro Restaurant,Paintball Field,Perfume Shop,Performing Arts Venue,Pedestrian Plaza
180,Itatiba,Brazilian Restaurant,Ice Cream Shop,Plaza,Pizza Place,Pharmacy,Hotel,Burger Joint,Café,Grocery Store,Food & Drink Shop
...,...,...,...,...,...,...,...,...,...,...,...
744,Taiacu,Ice Cream Shop,Zoo,Outdoor Sculpture,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store,Paintball Field
753,Santa Albertina,Bakery,Pharmacy,Ice Cream Shop,Pizza Place,Bar,Paintball Field,Perfume Shop,Performing Arts Venue,Pedestrian Plaza,Pastry Shop
762,Piacatu,Diner,Arts & Crafts Store,Ice Cream Shop,Outdoor Sculpture,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park,Paper / Office Supplies Store
764,Guaimbe,Bar,Farm,Restaurant,Ice Cream Shop,Outdoors & Recreation,Performing Arts Venue,Pedestrian Plaza,Pastry Shop,Pastelaria,Park
